In [2]:
from canvasapi import Canvas

In [51]:
CANVAS_URL = "https://canvas.liverpool.ac.uk"
CANVAS_TOKEN = ""

In [6]:
canvas = Canvas(CANVAS_URL, CANVAS_TOKEN)

In [7]:
course_id = 68788
assignment_id = 259524

In [8]:
assignment = canvas.get_course(course_id).get_assignment(assignment_id)

In [9]:
overrides = [x for x in assignment.get_overrides()]

In [12]:
len()

[AssignmentOverride(_requester=<canvasapi.requester.Requester object at 0x000001815D8850A0>, id=37178, assignment_id=259524, title=28 students, due_at=2024-08-09T16:00:00Z, due_at_date=2024-08-09 16:00:00+00:00, all_day=False, all_day_date=2024-08-09, all_day_date_date=2024-08-09 00:00:00+00:00, unlock_at=2024-08-05T08:00:00Z, unlock_at_date=2024-08-05 08:00:00+00:00, lock_at=2024-08-09T16:00:00Z, lock_at_date=2024-08-09 16:00:00+00:00, quiz_id=155653, unassign_item=False, student_ids=[9483, 73435, 103818, 110940, 111177, 112030, 131258, 132708, 135041, 135798, 136801, 137154, 137879, 139516, 140168, 140947, 141578, 141611, 141633, 141662, 141738, 141784, 142253, 142405, 143220, 144792, 146825, 146990], course_id=68788)]

In [17]:
user = canvas.get_user(73435, include=["profile"])

In [15]:
profile = user.get_profile()

In [24]:
profile

{'id': 73435,
 'name': 'Olivia Chaplin',
 'short_name': 'Olivia Chaplin',
 'sortable_name': 'Chaplin, Olivia',
 'avatar_url': 'https://canvas.liverpool.ac.uk/images/messages/avatar-50.png',
 'pronouns': 'She/Her',
 'title': None,
 'bio': None,
 'pronunciation': None,
 'primary_email': 'psochapl@liverpool.ac.uk',
 'login_id': 'psochapl@liverpool.ac.uk',
 'sis_user_id': '201590222psochapl',
 'integration_id': None,
 'time_zone': 'Europe/London',
 'locale': None}

In [21]:
import pandas as pd
from tqdm import tqdm

In [20]:
data = pd.read_csv("Assignment.csv")

In [46]:
def get_assignment_overrides(assignment):
    
    try:
        student_ids = []
        [student_ids.extend(x.student_ids) for x in assignment.get_overrides()]
    except:
        return None
    return student_ids

def get_student_profile(user_id):
    profile = canvas.get_user(user_id).get_profile()
    return profile

In [28]:
sample = data[data["assignment_id"] == 259524]

In [29]:
sample

,id,department,assignment_name,course,points_possible,assignment_id,unlock_at,lock_at,due_at,url,...,quiz,sas_exam,average_score,type,has_overrides,posted_at,rollover_to_course,previous_term_assignment,assignment_type,rubric_title
420,716213,School of Life Sciences,RESIT LIFE113.0 (60%) - Test 2,LIFE113-202324,100.0,259524,2024-08-05 08:00:00+00:00,2024-08-09 16:00:00+00:00,NaN,https://canvas.liverpool.ac.uk/courses/68788/a...,...,True,False,63.8,['online_quiz'],True,NaN,NaN,NaN,Online test 60 mins UG/PG,NaN


In [50]:


all_rows = []

# Iterate over each row in the data DataFrame
for i, row in tqdm(data.iterrows(), total=len(data)):

    try:
        df = pd.read_csv("assignment_overrides_resits_202324.csv")
    except:
        df = pd.DataFrame(columns=["course", "course_id", "assignment", "assignment_id", "sis_user_id", "sortable_name"])

    if len(df[df["assignment_id"] == row["assignment_id"]]) > 0:
        continue

    rows = []
    
    # Get the course using the course ID from the row
    course = canvas.get_course(row["course"], use_sis_id=True)
    
    try:
        # Get the assignment using the assignment ID from the row
        assignment = course.get_assignment(row["assignment_id"])
    except:
        continue
    
    # Get the assignment overrides using the get_assignment_overrides() function
    overrides = get_assignment_overrides(assignment)

    print(course.name, assignment.name, overrides)
    
    if overrides:
        # Iterate over each student ID in the overrides list
        for student_id in overrides:
            # Get the student profile using the student ID
            profile = get_student_profile(student_id)
            
            # Append the row data to the rows list
            rows.append({
                "course": course.name,
                "course_id": course.id,
                "assignment": assignment.name,
                "assignment_id": assignment.id,
                "sis_user_id": profile["sis_user_id"],
                "sortable_name": profile["sortable_name"]
            })
        
        new_df = pd.DataFrame(rows)

        # append new_df to df
        df = pd.concat([df, new_df])

        # Save the DataFrame to a CSV file
        df.to_csv("assignment_overrides_resits_202324.csv", index=False)


# Create a DataFrame from the all_rows list
df = pd.DataFrame(all_rows)

# Save the DataFrame to a CSV file
df.to_csv("assignment_overrides_resits_202324.csv", index=False)


  0%|          | 1/422 [00:01<12:45,  1.82s/it]

202324-LIFE107 - EXPERIMENTAL SKILLS IN CURRENT BIOLOGY RESIT LIFE107.0 (30%) LIFE107 Cycle 2 Assessment  21/22  []


  0%|          | 2/422 [00:02<07:22,  1.05s/it]

202324-LIFE107 - EXPERIMENTAL SKILLS IN CURRENT BIOLOGY RESIT LIFE107.1 (20%) Abstract submission link  []


  1%|          | 3/422 [00:02<05:45,  1.21it/s]

...

 99%|█████████▉| 418/422 [14:43<00:14,  3.73s/it]

202324-LIFE105 - GRAND CHALLENGES IN BIOLOGY RESIT LIFE105.2 (20%) - Explainer Video Assignment []
202324-LIFE103 - EVOLUTION RESIT Please submit a copy of your Comprehension text here for plagiarism checks [9483, 135798, 136801, 138128, 140931]


100%|█████████▉| 420/422 [14:49<00:06,  3.24s/it]

202324-LIFE101 - MOLECULES AND CELLS RESIT LIFE101.1 (40%) - Cycle 1 - Infographic []
202324-LIFE113 - Quantitative Skills for the Life Sciences RESIT LIFE113.1 (40%) - Test 1 [101, 112174, 135041, 135798, 140947, 141578, 141611, 141654, 141662, 141738, 142253, 143246, 144792, 145570, 146990]


100%|██████████| 422/422 [15:04<00:00,  2.14s/it]
